In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('gpcr_db_342.txt', encoding = 'unicode_escape')

In [3]:
# get UniProt, State, Function columns
sub_df = df[['UniProt', 'State', 'Function', 'PDB']]
uniprot_names = list(sub_df['UniProt'])
states = list(sub_df['State'])
functions = list(sub_df['Function'])
pdbids = list(sub_df['PDB'])

In [4]:
url = 'https://gpcrdb.org/services/structure/'
response = requests.get(url)
data = response.json()

In [5]:
data[1]

{'pdb_code': '5TGZ',
 'protein': 'cnr1_human',
 'family': '001_004_005_001',
 'species': 'Homo sapiens',
 'preferred_chain': 'A',
 'resolution': 2.8,
 'publication_date': '2016-11-02',
 'type': 'X-ray diffraction',
 'state': 'Inactive',
 'distance': None,
 'publication': 'https://dx.doi.org/10.1016/J.CELL.2016.10.004',
 'ligands': [{'name': 'AM6538',
   'type': 'small-molecule',
   'function': 'Antagonist',
   'PDB': 'ZDG',
   'SMILES': '[O-][N+](=O)OCCC#Cc1ccc(cc1)c1c(C)c(nn1c1ccc(cc1Cl)Cl)C(=O)NN1CCCCC1'}]}

In [6]:
crossdock_pdbs = []
ligands = []
for i in range(len(uniprot_names)):
    print(uniprot_names[i], states[i], functions[i], pdbids[i])
    matching_subdicts = []
    count = 0
    for subdict in data:
        # get gpcrdb structures whose uniprot name matches the current uniprot name
        if uniprot_names[i] in subdict['protein'].upper():
            count += 1
            matching_subdicts.append(subdict)

    # get pdbids, states, functions, resolutions of structures matching the uniprot code and create a dataframe
    sd_pdbs = []
    sd_states = []
    sd_functions = []
    sd_res = []
    
    # loop through structures matching the uniprot name
    for j in range(len(matching_subdicts)):
        sd_pdbs.append(matching_subdicts[j]['pdb_code'])
        sd_states.append(matching_subdicts[j]['state'])
        sd_res.append(matching_subdicts[j]['resolution'])
        ligand_data = matching_subdicts[j]['ligands']
        
        # if the structure has a ligand
        if len(ligand_data) > 0:
            sd_functions.append(ligand_data[0]['function'])
            
            # get ligand of sub_df entry
            if matching_subdicts[j]['pdb_code'] == pdbids[i]:
                print(matching_subdicts[j]['pdb_code'], ligand_data[0])
                
                # if theres a smiles key in the subdict (some structures don't have these)
                if len(ligand_data[0]) > 4:
                    ligands.append(ligand_data[0]['SMILES'])
                # otherwise, append nan
                else:
                    ligands.append(np.nan)
                
        # if the structure does not have a ligand
        else:
            sd_functions.append('NA')

    structure_df = pd.DataFrame({'Receptor' : uniprot_names[i], 'PDBID' : sd_pdbs, 'State': sd_states, 'Function' : sd_functions, 'Resolution' : sd_res})
    structure_df = structure_df.sort_values(by = 'Resolution')
    
    # get structures that match the state and function of the current sub_df entry and do not possess the same PDBID
    matching_df = structure_df.loc[(structure_df['State'] == states[i]) & (structure_df['Function'] == functions[i]) & (structure_df['PDBID'] != pdbids[i])]
    
    # if no structures match all criteria, don't match function
    if len(matching_df) == 0:
        matching_df = structure_df.loc[(structure_df['State'] == states[i]) & (structure_df['PDBID'] != pdbids[i])]
        
        # if dropping function doesn't work, drop state too
        if len(matching_df) == 0:
            matching_df = structure_df.loc[structure_df['PDBID'] != pdbids[i]]  
    
    # print structures matching criteria
    display(matching_df)
    
    # add pdbid of structure matching criteria to crossdock_pdbs
    if len(matching_df) > 0:
        crossdock_pdbs.append(list(matching_df['PDBID'])[0])
    
    # if there are no structures fitting the criteria, append NA (no cross docking for this entry)
    else:
        crossdock_pdbs.append(np.nan)
    
    print('\n')  

    # entries with no structure on GPCR DB
    if count == 0:
        print(uniprot_names[i])

ADRB2 Inactive Inverse agonist 2RH1
2RH1 {'name': 'Carazolol', 'type': 'small-molecule', 'function': 'Inverse agonist', 'PDB': 'CAU', 'SMILES': 'CC(C)NC[C@@H](COC1=CC=CC2=C1C3=CC=CC=C3N2)O'}


,Receptor,PDBID,State,Function,Resolution
19,ADRB2,5D5A,Inactive,Inverse agonist,2.48
15,ADRB2,3D4S,Inactive,Inverse agonist,2.80
9,ADRB2,3NY9,Inactive,Inverse agonist,2.84
8,ADRB2,3NY8,Inactive,Inverse agonist,2.84
23,ADRB2,5D6L,Inactive,Inverse agonist,3.20
21,ADRB2,5JQH,Inactive,Inverse agonist,3.20
22,ADRB2,5D5B,Inactive,Inverse agonist,3.80
16,ADRB2,4GBR,Inactive,Inverse agonist,3.99




ADRB1 Inactive Antagonist 2VT4
2VT4 {'name': '(S)-Cyanopindolol', 'type': 'small-molecule', 'function': 'Antagonist', 'PDB': 'P32', 'SMILES': 'CC(C)(C)NC[C@@H](COC1=CC=CC2=C1CC(=N2)C#N)O'}


,Receptor,PDBID,State,Function,Resolution
19,ADRB1,4BVN,Inactive,Antagonist,2.10
13,ADRB1,3ZPR,Inactive,Antagonist,2.70
1,ADRB1,3ZPQ,Inactive,Antagonist,2.80
3,ADRB1,2YCW,Inactive,Antagonist,3.00
9,ADRB1,2YCY,Inactive,Antagonist,3.15
11,ADRB1,2YCX,Inactive,Antagonist,3.25
20,ADRB1,5F8U,Inactive,Antagonist,3.35
6,ADRB1,2YCZ,Inactive,Antagonist,3.65




ADRB1 Inactive Agonist (partial) 2Y00
2Y00 {'name': 'Dobutamine', 'type': 'small-molecule', 'function': 'Agonist (partial)', 'PDB': 'Y00', 'SMILES': 'C[C@H](CCC1=CC=C(C=C1)O)NCCC2=CC(=C(C=C2)O)O'}


...

,Receptor,PDBID,State,Function,Resolution
0,GP139,7VUG,Active,Agonist,3.20
3,GP139,7VUH,Active,Agonist,3.22
1,GP139,7VUI,Active,Agonist,3.30




MRGX2 Active Agonist 7VV6
7VV6 {'name': 'compound 48/80', 'type': 'small-molecule', 'function': 'Agonist', 'PDB': '6IB', 'SMILES': 'CNCCc1cc(Cc2cc(CCNC)ccc2OC)c(c(c1)Cc1cc(CCNC)ccc1OC)OC'}


,Receptor,PDBID,State,Function,Resolution
1,MRGX2,7S8L,Active,Agonist,2.45
0,MRGX2,7S8M,Active,Agonist,2.54
7,MRGX2,7S8O,Active,Agonist,2.58
5,MRGX2,7VV5,Active,Agonist,2.76
11,MRGX2,7VUY,Active,Agonist,2.84
8,MRGX2,7VUZ,Active,Agonist,2.89
3,MRGX2,7VDH,Active,Agonist,2.90
4,MRGX2,7S8N,Active,Agonist,2.90
10,MRGX2,7VV4,Active,Agonist,2.97
12,MRGX2,7VV3,Active,Agonist,2.97




S1PR1 Active Agonist 7WF7
7WF7 {'name': 'sphingosine 1-phosphate', 'type': 'small-molecule', 'function': 'Agonist', 'PDB': 'S1P', 'SMILES': 'CCCCCCCCCCCCC/C=C/[C@H]([C@H](COP(=O)(O)O)N)O'}


,Receptor,PDBID,State,Function,Resolution
1,S1PR1,7TD4,Active,Agonist,2.60
5,S1PR1,7EO4,Active,Agonist,2.86
4,S1PR1,7EO2,Active,Agonist,2.89
10,S1PR1,7EVY,Active,Agonist,2.98
3,S1PR1,7TD3,Active,Agonist,3.00
6,S1PR1,7EVZ,Active,Agonist,3.07
9,S1PR1,7EW7,Active,Agonist,3.27
8,S1PR1,7EW0,Active,Agonist,3.42


In [7]:
len(ligands)

342

In [8]:
sub_df['crossdock_PDB'] = crossdock_pdbs
sub_df['dock_mol'] = ligands

C:\Users\Greg\AppData\Local\Temp/ipykernel_23752/1270241967.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['crossdock_PDB'] = crossdock_pdbs
C:\Users\Greg\AppData\Local\Temp/ipykernel_23752/1270241967.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['dock_mol'] = ligands


In [9]:
pd.set_option('display.max_rows', None)
sub_df

,UniProt,State,Function,PDB,crossdock_PDB,dock_mol
0,ADRB2,Inactive,Inverse agonist,2RH1,5D5A,CC(C)NC[C@@H](COC1=CC=CC2=C1C3=CC=CC=C3N2)O
1,ADRB1,Inactive,Antagonist,2VT4,4BVN,CC(C)(C)NC[C@@H](COC1=CC=CC2=C1CC(=N2)C#N)O
2,ADRB1,Inactive,Agonist (partial),2Y00,2Y01,C[C@H](CCC1=CC=C(C=C1)O)NCCC2=CC(=C(C=C2)O)O
3,ADRB1,Inactive,Agonist (partial),2Y01,2Y00,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...
4,ADRB1,Inactive,Agonist,2Y02,2Y03,COc1ccc(C[C@@H](C)NC[C@H](O)c2ccc(O)c3[nH]c(=O...
5,ADRB1,Inactive,Agonist,2Y03,2Y02,CC(C)NC[C@H](O)c1ccc(O)c(O)c1
6,ADRB1,Inactive,Agonist (partial),2Y04,2Y00,OCc1cc(ccc1O)[C@H](CNC(C)(C)C)O
7,ADRB1,Inactive,Antagonist,2YCW,4BVN,CC(C)NC[C@@H](COC1=CC=CC2=C1C3=CC=CC=C3N2)O
8,ADRB1,Inactive,Antagonist,2YCY,4BVN,CC(C)(C)NC[C@@H](COC1=CC=CC2=C1CC(=N2)C#N)O
9,ADRB1,Inactive,Antagonist,2YCZ,4BVN,CC(C)(C)NC[C@@H](COC1=CC=CC2=C1C(=C(N2)C#N)I)O


In [21]:
sub_df_non_NA = sub_df[sub_df['crossdock_PDB'].notnull()]

In [22]:
len(sub_df_non_NA) # number of entries to cross dock into

331

In [24]:
sub_df_non_NA.to_csv('crossdock_pdbs.txt', index_label = 'Index')